In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk
from dotenv import load_dotenv

import os

In [2]:
# Database connection details
load_dotenv()

username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host =  os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database =  os.getenv("DB_DATABASE")

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

In [3]:
query = text("SELECT * FROM Perfumes_v2") 

with engine.connect() as connection:
    result = connection.execute(query)
    perfumes = [dict(row._mapping) for row in result]

df = pd.DataFrame(perfumes)
df

,id,Name,Designer,URL,Gender,Accords,Description,ProsAndCons,TopNotes,MiddleNotes,BaseNotes,Longevity,Sillage
0,1,Unicorn Milkshake,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,sweet (100%); vanilla (58.1656%); lactonic (55...,Unicorn Milkshake by The Dua Brand is a Floral...,,[],[],[],,
1,2,Princeless Princess,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,fresh (100%); fresh spicy (82.6144%); sweet (8...,Princeless Princess by The Dua Brand is a Orie...,,[],[],[],,
2,3,White Milk,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,vanilla (100%); musky (86.7568%); sweet (79.18...,White Milk by The Dua Brand is a fragrance for...,,[],[],[],,
3,4,Popped Cherry,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,almond (100%); sweet (82.4996%); vanilla (79%)...,Popped Cherry by The Dua Brand is a Aromatic F...,,[],[],[],,
4,5,Angelic Elixir,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,warm spicy (100%); woody (91.3583%); cinnamon ...,Angelic Elixir by The Dua Brand is a Aromatic ...,,[],[],[],,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,3028,Maze Eau de Parfum,Al Haramain Perfumes,https://www.fragrantica.com/perfume/Al-Haramai...,women and men,citrus (100%); woody (94.503%); aromatic (92.4...,Maze Eau de Parfum by Al Haramain Perfumes is ...,,"['Rose', 'Bergamot', 'Vermouth', 'Lemon', 'Dav...","['Rose', 'Sandalwood', 'Saffron', 'Clove', 'Ja...","['Sandalwood', 'Patchouli', 'Musk', 'Fruity No...",,
3028,3029,Musk Black Vanilla,Al Haramain Perfumes,https://www.fragrantica.com/perfume/Al-Haramai...,women and men,amber (100%); smoky (65.7447%); vanilla (64.75...,Musk Black Vanilla by Al Haramain Perfumes is ...,,"['Black Currant', 'Cedar', 'Herbal Notes']","['Vanilla', 'Amber', 'Spices']","['Incense', 'Patchouli', 'Musk']",,
3029,3030,Tanasuk (2019),Al Haramain Perfumes,https://www.fragrantica.com/perfume/Al-Haramai...,women and men,sweet (100%); vanilla (72.6594%); amber (68.78...,Tanasuk (2019) by Al Haramain Perfumes is a Or...,,"['Grenadine', 'Saffron', 'Rose']","['Amber', 'Woody Notes', 'Jasmine']","['Brown sugar', 'Vanilla', 'Musk']",,
3030,3031,Firdous,Al Haramain Perfumes,https://www.fragrantica.com/perfume/Al-Haramai...,women and men,white floral (100%); mossy (78.6112%); woody (...,Firdous by Al Haramain Perfumes is a Chypre fr...,,"['Moss', 'Lily-of-the-Valley', 'Lily', 'Musk',...",[],[],,


In [4]:
query = text("SELECT Accords FROM Perfumes_v2") 

with engine.connect() as connection:
    result = connection.execute(query)
    perfumes = [dict(row._mapping) for row in result]

df = pd.DataFrame(perfumes)

accord_counter = Counter()

for row in df['Accords'].dropna():
    matches = re.findall(r'([\w\s\-]+)\s*\(\d', row)
    for accord in matches:
        accord_counter[accord.strip()] += 1

accord_freq_df = pd.DataFrame(accord_counter.items(), columns=["Accord", "Count"]).sort_values(by="Count", ascending=False)

accord_freq_df.to_excel("accord_frequencies_2.xlsx", index=False)


In [ ]:
query = text("SELECT Description FROM Perfumes") 

nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

with engine.connect() as connection:
    result = connection.execute(query)
    perfumes = [dict(row._mapping) for row in result]

word_counter = Counter()

for desc in df['Description'].dropna():
    words = re.findall(r'\b\w+\b', desc.lower())
    filtered_words = [word for word in words if word not in stop_words]
    word_counter.update(filtered_words)

word_freq_df = pd.DataFrame(word_counter.items(), columns=["Word", "Count"]).sort_values(by="Count", ascending=False)

word_freq_df.to_excel("description_word_counts_filtered.xlsx", index=False)

print(word_freq_df.head(10))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jakubwasik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


           Word  Count
41        notes   2506
7     fragrance   2177
86      perfume   1028
11     launched    901
95    português    859
89      español    859
102  українська    859
101     العربية    859
100      română    859
96     ελληνικά    859
